# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kahului,20.8947,-156.4700,24.36,85,40,5.81,US,1675621019
1,1,koppang,61.5706,11.0555,-11.63,92,67,1.64,NO,1675621178
2,2,hobart,-42.8794,147.3294,14.47,76,94,2.57,AU,1675620921
3,3,thompson,55.7435,-97.8558,-16.91,78,75,1.54,CA,1675620945
4,4,moratuwa,6.7730,79.8816,24.27,86,98,3.88,LK,1675621179


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_data = city_data_df.hvplot.points("Lng", "Lat", geo = True, size = "Humidity", scale = 1, color = "City", alpha = 0.5, tiles = "OSM", frame_width = 700, frame_height = 500)

# Display the map
map_data

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
narrow_city_df = narrow_city_df.dropna()

# Display sample data
narrow_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,east london,-33.0153,27.9116,24.52,88,0,4.12,ZA,1675620802
207,207,ixtapa,20.7000,-105.2000,24.57,65,0,3.09,MX,1675621183
283,283,santa cruz del norte,23.1539,-81.9236,26.97,60,0,2.78,CU,1675621261
401,401,salalah,17.0151,54.0924,24.05,69,0,1.54,OM,1675621243
445,445,san patricio,28.0170,-97.5169,24.92,64,0,2.50,US,1675621323


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = narrow_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
47,east london,ZA,-33.0153,27.9116,88,
207,ixtapa,MX,20.7000,-105.2000,65,
283,santa cruz del norte,CU,23.1539,-81.9236,60,
401,salalah,OM,17.0151,54.0924,69,
445,san patricio,US,28.0170,-97.5169,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    print(name_address.url)
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
#hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=24bef6b3488a4c63bd024e90e3da8c87&limit=20&filter=circle%3A27.9116%2C+-33.0153%2C+10000&bias=proximity%3A27.9116%2C+-33.0153
east london - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=24bef6b3488a4c63bd024e90e3da8c87&limit=20&filter=circle%3A-105.2%2C+20.7%2C+10000&bias=proximity%3A-105.2%2C+20.7
ixtapa - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=24bef6b3488a4c63bd024e90e3da8c87&limit=20&filter=circle%3A-81.9236%2C+23.1539%2C+10000&bias=proximity%3A-81.9236%2C+23.1539
santa cruz del norte - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=24bef6b3488a4c63bd024e90e3da8c87&limit=20&filter=circle%3A54.0924%2C+17.0151%2C+10000&bias=proximity%3A54.0924%2C+17.0151
salalah - nearest hotel: No hotel found
https://api.geoapify.com/v

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)